# Sequence to function

## The problem
The problem we are working on is called sequence to function. Ideally this means we have a sequence and we infer some kind of function

### The sequence: 
To me this means a few different things. 
- Gene sequence mutations
- Gene and Protein Orthologs
- Post-translational modification

### The function: 
This also could mean a few things. My mind initially went to physiology, but now realized function could be alot broader. For example simple ligand enzyme binding could be related to funciton. The pathway that the protein is involved in, the regulation
- A change in ligand binding interactions (Initial GPCR activation)
- A change in metabolitic secondary activity (GPCRs downstream)

## The solution
When thinking about a solution to the hackathon problem. Ryan has suggested, could we start with building a full profile with only two data categories (for the sequence). To me this means: 
- Gene name, mutation/ortholog
- Protein name, post-translational modification

Given those could we pull out somekind of knowledge graph that will allow us to relate it to currently known research. This is where we will need the __agent__

In [ ]:
import sys, os
sys.path.append("..")

from scripts.fetch_data import fetch_uniprot_data, split_colon_list
from scripts.reference_scoring import (
    collect_reference_network_for_genes,
    score_reference_dataframe,
    select_top_scoring_articles,
    attach_full_text_columns,
)
from scripts.epmc_utils import save_dataframe_rows_as_json
import pandas as pd


In [4]:
#This simple script creates the JSON directory if it doesn't exist. We are adding this to .gitignore, maybe it is too heavy?
if not os.path.isdir("../data/papers"):
    os.makedirs("../data/papers", exist_ok = True)
    print("Making a new directory")
else:
    print("Directory already exists")

Directory already exists


In [5]:
# Maybe here can go the parsing terms
uniprot_data = fetch_uniprot_data(["CCR2"])
display(uniprot_data)
citation_list = split_colon_list(uniprot_data.citation_titles[0])

,gene_symbol,uniprot_id,protein_name,sequence,pmids,dois,citation_titles,reviewed
0,CCR2,P41597,C-C chemokine receptor type 2,MLSTSRSRFIRNTNESGEEVTTFFDYDYGAPCHKFDVKQIGAQLLP...,None,None,Molecular cloning and functional expression of...,False


In [ ]:
reference_network = collect_reference_network_for_genes(
    uniprot_df=uniprot_data,
    include=("references", "citations"),
    max_depth=1,
    delay=0.1,
)

if reference_network.empty:
    print("No reference or citation records were returned.")
else:
    print(f"Network size: {len(reference_network)} articles")
    display(reference_network.head())


In [ ]:
if reference_network.empty:
    scored_network = reference_network
    print("Skipping scoring step; network is empty.")
else:
    scored_network = score_reference_dataframe(
        reference_network,
        weights={"year": 0.4, "function": 0.35, "longevity": 0.25},
        delay=0.1,
        include_fulltext=False,
    )
    display(scored_network[[
        "gene_symbol",
        "relation_primary",
        "title",
        "year",
        "composite_score",
    ]].head(10))


In [ ]:
n_references = 10

if scored_network.empty:
    top_references = scored_network
    print("No articles available for ranking or export.")
else:
    top_references = select_top_scoring_articles(
        scored_network,
        n_per_gene=n_references,
        relation_filter=("reference", "citation"),
    )
    top_references = attach_full_text_columns(
        top_references,
        delay=0.1,
        include_xml=True,
    )
    preview = top_references.assign(
        abstract_chars=top_references["abstract_text"].fillna("").str.len(),
        full_text_chars=top_references["full_text"].fillna("").str.len(),
    )
    display(preview[[
        "gene_symbol",
        "relation_primary",
        "title",
        "year",
        "composite_score",
        "abstract_chars",
        "full_text_chars",
    ]])

    saved_files = save_dataframe_rows_as_json(
        top_references,
        "../data/papers",
        id_column="PMID",
        filename_prefix="PMID",
        indent=4,
        drop_missing=True,
    )
    print(f"Saved {len(saved_files)} top-scoring JSON files to ../data/papers")


In [ ]:
#Now once I have build a huge 